# get_dummies() 

`get_dummies()`를 활용하여 범주형 자료를 더미화하고, 아래의 문제가 있을 때 `get_dummies()`는 어떤 결과를 내놓는지 확인해보자. 

> 1. numeric value가 string 형태로 들어가있을 때.  
> 2. missing value가 있을 때. 
> 3. 다른 데이터셋에 대입할 때.

***

#### 데이터 만들기 

In [2]:
import pandas as pd
 
data = {'Wealth':[27, 27, 22, 32],
        'bld':['A', 'B', 'A', 'O'],
        'Qual':['Msc', 'MA', 'MA', 'Msc']}
 

data = pd.DataFrame(data, index = ['Jai', 'Princi', 'Salah', 'Anuj'])

In [3]:
data

,Wealth,bld,Qual
Jai,27,A,Msc
Princi,27,B,MA
Salah,22,A,MA
Anuj,32,O,Msc


***

### 더미화 

In [4]:
dummy = pd.get_dummies(data)
dummy

,Wealth,bld_A,bld_B,bld_O,Qual_MA,Qual_Msc
Jai,27,1,0,0,0,1
Princi,27,0,1,0,1,0
Salah,22,1,0,0,1,0
Anuj,32,0,0,1,0,1


***

## numeric value가 string으로 기입되어 있는 경우  

In [5]:
data = {'Wealth':['27', '27', '22', '32'],
        'bld':['A', 'B', 'A', 'O'],
        'Qual':['Msc', 'MA', 'MA', 'Msc']}

data = pd.DataFrame(data, index = ['Jai', 'Princi', 'Salah', 'Anuj'])
dummy = pd.get_dummies(data)
dummy

,Wealth_22,Wealth_27,Wealth_32,bld_A,bld_B,bld_O,Qual_MA,Qual_Msc
Jai,0,1,0,1,0,0,0,1
Princi,0,1,0,0,1,0,1,0
Salah,1,0,0,1,0,0,1,0
Anuj,0,0,1,0,0,1,0,1


> 완전한 수치형 자료임에도 불구하고  불필요하게 재산(wealth)가 더미변수화 되었다.

***

## missing value가 있는 경우  

In [6]:
data = {'Wealth':[27, 27, 22, 32],
        'bld':['A', None, 'A', 'O'],
        'Qual':['Msc', 'MA', 'MA', 'Msc']}

data = pd.DataFrame(data, index = ['Jai', 'Princi', 'Salah', 'Anuj'])
dummy = pd.get_dummies(data)
dummy

,Wealth,bld_A,bld_O,Qual_MA,Qual_Msc
Jai,27,1,0,0,1
Princi,27,0,0,1,0
Salah,22,1,0,1,0
Anuj,32,0,1,0,1


보기에는 문제가 없으나, 더미변수화를 하고 다중공선성 문제를 해결하기 위해 `drop_first=True`를 하는 순간 문제가 생긴다.

In [7]:
dummy = pd.get_dummies(data, drop_first = True)
dummy

,Wealth,bld_O,Qual_Msc
Jai,27,0,1
Princi,27,0,0
Salah,22,0,0
Anuj,32,1,1


#### Princi의 혈액형은 분명 기입되지 않았지만, drop을 하고 난 뒤에는 Jai, Anuj와 같은 혈액형으로 읽혀지고 있다.

***

## 다른 데이터셋에 적용할 경우  

In [9]:
data = {'Wealth':[27, 27, 22, 32],
        'bld':['A', 'AB', 'A', 'O'],
        'Qual':['Msc', 'MA', 'MA', 'Msc']}

train = pd.DataFrame(data, index = ['Jai', 'Princi', 'Salah', 'Anuj'])

In [10]:
data = {'Wealth':[51, 27, 22, 32],
        'bld':['A', 'B', 'AB', 'O'],
        'Qual':['MB', 'MA', 'MA', 'Msc']}

test = pd.DataFrame(data, index = ['Hwan', 'Jason', 'Lee', 'Andrew'])

In [12]:
train

,Wealth,bld,Qual
Jai,27,A,Msc
Princi,27,AB,MA
Salah,22,A,MA
Anuj,32,O,Msc


In [13]:
test

,Wealth,bld,Qual
Hwan,51,A,MB
Jason,27,B,MA
Lee,22,AB,MA
Andrew,32,O,Msc


현재 train과 달리 test 데이터에는 B와 MB 같은 새로운 데이터셋이 추가되었다.

In [14]:
dummy1 = pd.get_dummies(train)
dummy2 = pd.get_dummies(test)

In [15]:
dummy1

,Wealth,bld_A,bld_AB,bld_O,Qual_MA,Qual_Msc
Jai,27,1,0,0,0,1
Princi,27,0,1,0,1,0
Salah,22,1,0,0,1,0
Anuj,32,0,0,1,0,1


In [16]:
dummy2

,Wealth,bld_A,bld_AB,bld_B,bld_O,Qual_MA,Qual_MB,Qual_Msc
Hwan,51,1,0,0,0,0,1,0
Jason,27,0,0,1,0,1,0,0
Lee,22,0,1,0,0,1,0,0
Andrew,32,0,0,0,1,0,0,1


새로운 혈액형이 추가되면서 아예 컬럼 갯수가 달라졌다. 따라서 train 데이터를 활용해서 모델을 피팅하면 나중에 test에서 사용할 수 없게 된다.